In [221]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import random
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader
import cv2

import PIL.Image as Image
from torchvision.transforms import ToTensor, ToPILImage
import torch.nn.functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [222]:
def read_video(filename, num_of_frames=100):
    """input: 
            Video name - имя файла в текущей папке, 
            Num_of_frames - число фрэймов
       output: 
            FloatTensor [T, C, H, W]
    """
    return torchvision.io.read_video(filename, pts_unit='sec')[0][:num_of_frames].permute(0,3,1,2).type(torch.FloatTensor)

def write_video(filename, video_array, fps=1):
    """Записывает тензор в формате [T, C, H, W] в видеофайл.
       input:
           filename - путь для save видео
           video_array - тензор [T, C, H, W]
           fps - число кадров в секунду
    """
    torchvision.io.write_video(filename, video_array.permute(0,2,3,1).type(torch.uint8), fps)
    
def show_frame(input_tensor, title=''):
    """Функция для отрисовки кадров.
       input:
           Любой тензор [1,C,H,W]
    """
    image = input_tensor.permute(1, 2, 0).numpy()
    plt.imshow((image).astype(np.uint8))
    plt.title(title)
    plt.show()
    
def set_random_seed(seed=0):
    """Сделаем систему детерминированной."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def get_device():
    """Если система поддерживает CUDA, то вернет GPU
       иначе вернет CPU. Нужно для закидывания модели и тензоров в GPU.
       Потом можно к любому тензору, модели применять .to(get_device())"""
    global device
    return device

In [232]:
set_random_seed(0)
video = read_video('test.mp4')
video.shape

torch.Size([100, 3, 1080, 1920])

In [207]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(get_device())
model.eval();

In [ ]:
# Тут у меня в общем ошибка у rectangle, потом гляну почему так
# Еще потом реализую класс Dataset, чтобы его передать в Dataloader. Это надо, чтобы создать батчи для трэйн.тест.
# Но пока думаю нет смысла в батчевании, потом когда дадут датасет.
# Потом я нашел туториал, где можно fasterrcnn_resnet50_fpn докрутить

In [233]:
def detect_obj(image):
#     tensor_image = torch.tensor(image)
#     tensor_image = tensor_image.permute(2, 0, 1)
    img_01 = image / 255.
    model.eval()
    predictions = model([img_01])
#     print(predictions[0]['scores'])
#     print(predictions[0]['boxes'][0])
    if predictions[0]['scores'][0] > 0.5:
        box = predictions[0]['boxes'][0]
        point1 = (round(box[0].item()), round(box[1].item()))
        point2 = (round(box[2].item()), round(box[3].item()))
#         print(point1)
#         print(point2)
        image = cv2.rectangle(ToPILImage(image), (int(point1[0]), int(point1[1])), (int(point2[0]), int(point2[1])), (255,0,0), 2)
    return image


detect_obj(video[0])

TypeError: an integer is required (got type tuple)